In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import math

In [2]:
train_dir = 'F:/Desktop/TF图集/flower_train_1/output/'

L1 = []
L2 = []
L3 = []
L4 = []
L5 = []
label_1 = []
label_2 = []
label_3 = []
label_4 = []
label_5 = []

In [3]:
#获取（处理后64x64）图片路径
#ratio，测试集比率
def get_files(file_dir, ratio):
    for file in os.listdir(file_dir + '/1'):
        L1.append(file_dir + '/1/' +file)
        label_1.append(0)
    for file in os.listdir(file_dir + '/2'):
        L2.append(file_dir + '/2/' +file)
        label_2.append(1)
    for file in os.listdir(file_dir + '/3'):
        L3.append(file_dir + '/3/' +file)
        label_3.append(2)
    for file in os.listdir(file_dir + '/4'):
        L4.append(file_dir + '/4/' +file)
        label_4.append(3)
    for file in os.listdir(file_dir + '/5'):
        L5.append(file_dir + '/5/' +file)
        label_5.append(4)
    #hstack 水平叠堆数组
    #均在数组中使用hstack
    image_list = np.hstack((L1, L2, L3, L4, L5))
    label_list = np.hstack((label_1, label_2, label_3, label_4, label_5))
    
    
    #shuffle 打乱顺序
    order1 = np.array([image_list, label_list])
    #transpose 转置
    order2 = order1.transpose()
    np.random.shuffle(order2)
    
    print(order1)
    print(order2)
#    print(disorder)
    for x in L1():
        print(x)
    #disorder转list
    image_list_turn = list(order2[:,0])
    label_list_turn = list(order2[:,1])
    
    #划分list为test & train
    n_sample = len(label_list_turn)
    n_test = int(math.ceil(n_sample * ratio))
    n_train = n_sample - n_test
    
    train_images = image_list_turn[0:n_train]
    train_labels = label_list_turn[0:n_train]
    train_labels = [int(float(i)) for i in train_labels]
    test_images = image_list_turn[n_train:-1]
    test_labels = label_list_turn[n_train:-1]
    test_labels = [int(float(i)) for i in test_labels]
    
    return train_images, train_labels, test_images, test_labels

In [4]:
#生成batch
#step1：将上面生成的List传入get_batch() ，转换类型，产生一个输入队列queue，因为img和lab
#是分开的，所以使用tf.train.slice_input_producer()，然后用tf.read_file()从队列中读取图像
#   image_W, image_H, ：设置好固定的图像高度和宽度
#   设置batch_size：每个batch要放多少张图片
#   capacity：一个队列最大多少
def get_batch(image, label, image_W, image_H, batch_size, capacity):
    #转换类型
    image = tf.cast(image, tf.string)
    label = tf.cast(label, tf.int32)
 
    # make an input queue
    input_queue = tf.train.slice_input_producer([image, label])
 
    label = input_queue[1]
    image_contents = tf.read_file(input_queue[0]) #read img from a queue  
    
#step2：将图像解码，不同类型的图像不能混在一起，要么只用jpeg，要么只用png等。
    image = tf.image.decode_jpeg(image_contents, channels=3) 
    
#step3：数据预处理，对图像进行旋转、缩放、裁剪、归一化等操作，让计算出的模型更健壮。
    image = tf.image.resize_image_with_crop_or_pad(image, image_W, image_H)
    image = tf.image.per_image_standardization(image)
    
#step4：生成batch
#image_batch: 4D tensor [batch_size, width, height, 3],dtype=tf.float32 
#label_batch: 1D tensor [batch_size], dtype=tf.int32
    image_batch, label_batch = tf.train.batch([image, label],
                                                batch_size= batch_size,
                                                num_threads= 32, 
                                                capacity = capacity)
    #重新排列label，行数为[batch_size]
    label_batch = tf.reshape(label_batch, [batch_size])
    image_batch = tf.cast(image_batch, tf.float32)
    return image_batch, label_batch

In [5]:
def inference(images, batch_size, n_classes):
#一个简单的卷积神经网络，卷积+池化层x2，全连接层x2，最后一个softmax层做分类。
#卷积层1
#64个3x3的卷积核（3通道），padding=’SAME’，表示padding后卷积的图与原图尺寸一致，激活函数relu()
    with tf.variable_scope('conv1') as scope:
        
        weights = tf.Variable(tf.truncated_normal(shape=[3,3,3,64], stddev = 1.0, dtype = tf.float32), 
                              name = 'weights', dtype = tf.float32)
        
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [64]),
                             name = 'biases', dtype = tf.float32)
        
        conv = tf.nn.conv2d(images, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name= scope.name)
        
#池化层1
#3x3最大池化，步长strides为2，池化后执行lrn()操作，局部响应归一化，对训练有利。
    with tf.variable_scope('pooling1_lrn') as scope:
        pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME', name='pooling1')
        '''tf.nn.max_pool(value, ksize, strides, padding, name=None)
            value :conv1 一般池化层接在卷积层后面，所以输入通常是feature map，依然是[batch, height, width, channels]这样的shape
            ksize :因为我们不想在batch和channels上做池化，所以这两个维度设为了1
            strides :窗口在每一个维度上滑动的步长，一般也是[1, stride,stride, 1]
            padding : SAME / VALID ,0补全/不补全
            返回 feature map
            池化目的：筛选最大值，不改变矩阵形状
        '''
        norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='norm1')
        
#卷积层2
#16个3x3的卷积核（16通道），padding=’SAME’，表示padding后卷积的图与原图尺寸一致，激活函数relu()
    with tf.variable_scope('conv2') as scope:
        weights = tf.Variable(tf.truncated_normal(shape=[3,3,64,16], stddev = 0.1, dtype = tf.float32), 
                              name = 'weights', dtype = tf.float32)
        
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [16]),
                             name = 'biases', dtype = tf.float32)
        
        conv = tf.nn.conv2d(norm1, weights, strides = [1,1,1,1],padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name='conv2')
        
#池化层2
#3x3最大池化，步长strides为2，池化后执行lrn()操作，
    #pool2 and norm2
    with tf.variable_scope('pooling2_lrn') as scope:
        norm2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001/9.0,beta=0.75,name='norm2')
        pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,1,1,1],padding='SAME',name='pooling2')
        
#全连接层3
#128个神经元，将之前pool层的输出reshape成一行，激活函数relu()
    with tf.variable_scope('local3') as scope:
        reshape = tf.reshape(pool2, shape=[batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = tf.Variable(tf.truncated_normal(shape=[dim,128], stddev = 0.005, dtype = tf.float32),
                             name = 'weights', dtype = tf.float32)
        
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [128]), 
                             name = 'biases', dtype=tf.float32)
        
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
        
#全连接层4
#128个神经元，激活函数relu() 
    with tf.variable_scope('local4') as scope:
        weights = tf.Variable(tf.truncated_normal(shape=[128,128], stddev = 0.005, dtype = tf.float32),
                              name = 'weights',dtype = tf.float32)
        
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [128]),
                             name = 'biases', dtype = tf.float32)
        
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name='local4')
        
#dropout层        
#    with tf.variable_scope('dropout') as scope:
#        drop_out = tf.nn.dropout(local4, 0.8)
            
        
#Softmax回归层
#将前面的FC层输出，做一个线性回归，计算出每一类的得分，在这里是2类，所以这个层输出的是两个得分。
    with tf.variable_scope('softmax_linear') as scope:
        weights = tf.Variable(tf.truncated_normal(shape=[128, n_classes], stddev = 0.005, dtype = tf.float32),
                              name = 'softmax_linear', dtype = tf.float32)
        
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [n_classes]),
                             name = 'biases', dtype = tf.float32)
        
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name='softmax_linear')
 
    return softmax_linear

In [6]:
#loss计算
    #传入参数：logits，网络计算输出值。labels，真实值，在这里是0或者1
    #返回参数：loss，损失值
def losses(logits, labels):
    with tf.variable_scope('loss') as scope:
        cross_entropy =tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        tf.summary.scalar(scope.name+'/loss', loss)
    return loss

In [7]:
#loss损失值优化
    #输入参数：loss。learning_rate，学习速率。
    #返回参数：train_op，训练op，这个参数要输入sess.run中让模型去训练。
def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op

In [8]:
#评价/准确率计算
    #输入参数：logits，网络计算值。labels，标签，也就是真实值，在这里是0或者1。
    #返回参数：accuracy，当前step的平均准确率，也就是在这些batch中多少张图片被正确分类了。
def evaluation(logits, labels):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        tf.summary.scalar(scope.name+'/accuracy', accuracy)
    return accuracy

In [9]:
#变量声明
N_CLASSES = 5  #husky,jiwawa,poodle,qiutian
IMG_W = 64   # resize图像，太大的话训练时间久
IMG_H = 64
BATCH_SIZE =64
CAPACITY = 200
MAX_STEP = 900 # 一般大于10K
learning_rate = 0.0001 # 一般小于0.0001
 
#获取批次batch
train_dir = 'F:/Desktop/TF图集/flower_train_1/output/'   #训练样本的读入路径
logs_train_dir = 'F:/Desktop/TF图集/flower_train_1/log/'    #logs存储路径

#train, train_label = input_data.get_files(train_dir)
train, train_label, val, val_label = get_files(train_dir, 0.3)
#训练数据及标签
train_batch,train_label_batch = get_batch(train, train_label, IMG_W, IMG_H, BATCH_SIZE, CAPACITY)
#测试数据及标签
val_batch, val_label_batch = get_batch(val, val_label, IMG_W, IMG_H, BATCH_SIZE, CAPACITY) 
 
#训练操作定义
train_logits = inference(train_batch, BATCH_SIZE, N_CLASSES)
train_loss = losses(train_logits, train_label_batch)        
train_op = trainning(train_loss, learning_rate)
train_acc = evaluation(train_logits, train_label_batch)
 
#测试操作定义
test_logits = inference(val_batch, BATCH_SIZE, N_CLASSES)
test_loss = losses(test_logits, val_label_batch)        
test_acc = evaluation(test_logits, val_label_batch)
 
#这个是log汇总记录
summary_op = tf.summary.merge_all() 
 
#产生一个会话
with tf.Session() as sess:
    #产生一个writer来写log文件
    train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph) 
    #val_writer = tf.summary.FileWriter(logs_test_dir, sess.graph) 
    #产生一个saver来存储训练好的模型
    saver = tf.train.Saver()
    #所有节点初始化
    sess.run(tf.global_variables_initializer())  
    #队列监控
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    #进行batch的训练
    try:
        #执行MAX_STEP步的训练，一步一个batch
        for step in np.arange(MAX_STEP):
            if coord.should_stop():
                break
            #启动以下操作节点，有个疑问，为什么train_logits在这里没有开启？
            _, tra_loss, tra_acc = sess.run([train_op, train_loss, train_acc])

            #每隔50步打印一次当前的loss以及acc，同时记录log，写入writer   
            if step % 10  == 0:
                print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, tra_loss, tra_acc*100.0))
                summary_str = sess.run(summary_op)
                train_writer.add_summary(summary_str, step)
            #每隔100步，保存一次训练好的模型
            if (step + 1) == MAX_STEP:
                checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=step)

    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')

    finally:
        coord.request_stop()

Step 0, train loss = 1.61, train accuracy = 18.75%
Step 10, train loss = 1.59, train accuracy = 25.00%
Step 20, train loss = 1.56, train accuracy = 26.56%
Step 30, train loss = 1.49, train accuracy = 46.88%
Step 40, train loss = 1.38, train accuracy = 40.62%
Step 50, train loss = 1.18, train accuracy = 48.44%
Step 60, train loss = 1.24, train accuracy = 46.88%
Step 70, train loss = 1.04, train accuracy = 59.38%
Step 80, train loss = 1.10, train accuracy = 56.25%
Step 90, train loss = 0.99, train accuracy = 56.25%
Step 100, train loss = 1.09, train accuracy = 56.25%
Step 110, train loss = 1.22, train accuracy = 43.75%
Step 120, train loss = 1.12, train accuracy = 56.25%
Step 130, train loss = 1.17, train accuracy = 65.62%
Step 140, train loss = 1.10, train accuracy = 51.56%
Step 150, train loss = 1.15, train accuracy = 53.12%
Step 160, train loss = 1.21, train accuracy = 45.31%
Step 170, train loss = 1.02, train accuracy = 62.50%
Step 180, train loss = 1.00, train accuracy = 51.56%
Step